# Chicago

In [1]:
import pandas as pd
import numpy as np
import re

# Crawling
from bs4 import BeautifulSoup
from selenium import webdriver
from urllib.request import urlopen
import urllib.request
import urllib.parse
    
# conda install -c conda-forge tqdm
from tqdm import tqdm
from tqdm import tqdm_notebook
import time

## 3-3 실전: 시카고 샌드위치 맛집 소개 사이트에 접근하기

In [111]:
import requests
url = 'https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-Chicago/'
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.87 Safari/537.36'}
req = requests.get(url, headers = header)
soup = BeautifulSoup(req.text, 'html.parser')

listdata = []

for i, row in enumerate(soup.select('#page .sammy')):

    listdata.append({})

    # Rank
    elRank = row.select_one('.sammyRank')
    elRankText = int(elRank.get_text(strip=True))
    listdata[i].update({'rank':elRankText})

    # Listing
    elListing = row.select_one('.sammyListing a')
    elListing.em.extract()

    # LIST
    tmpArr = []

    # Menu
    elMenu = elListing.select_one('b')
    elMenuText = elMenu.get_text(strip=True)
    listdata[i].update({'menu':elMenuText})
    tmpArr.append(elMenuText)

    # Cafe
    elCafe = elListing
    elCafe.b.extract()
    elCafeText = elCafe.get_text(strip=True)
    listdata[i].update({'cafe':elCafeText})
    tmpArr.append(elCafeText)

    # Title
    tmpArr.reverse()
    elTitleText = ' '.join(tmpArr)
    listdata[i].update({'title':elTitleText})

    # Link
    elLink = elListing
    elLinkBase = 'https://www.chicagomag.com'
    elLinkText = urllib.parse.urljoin(base=elLinkBase, url=elLink.get('href'))
    listdata[i].update({'link':elLinkText})

listdata[:2]

[{'rank': 1,
  'menu': 'BLT',
  'cafe': 'Old Oak Tap',
  'title': 'Old Oak Tap BLT',
  'link': 'https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/'},
 {'rank': 2,
  'menu': 'Fried Bologna',
  'cafe': 'Au Cheval',
  'title': 'Au Cheval Fried Bologna',
  'link': 'https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Au-Cheval-Fried-Bologna/'}]

In [169]:
df = pd.DataFrame(listdata)
df.to_csv('04_chicago_list.csv', sep=',', encoding='utf8')

## 3-4 다수의 웹 페이지에 자동으로 접근해서 원하는 정보 가져오기

In [100]:
tmp_html = urlopen(df['link'][0])
tmp_soup = BeautifulSoup(tmp_html, 'html.parser')

viewdata = []

viewdata.append({})

# Title
elTitle = tmp_soup.select_one('#article-header-image-container .headline')
elTitleText = elTitle.get_text(strip=True)
elTitleText = re.sub('^\d+\.', '', elTitleText).strip()
viewdata[0].update({'title': elTitleText})

# Addy
elAddy = tmp_soup.select_one('.addy')

text = elAddy.em.get_text(strip=True).split(',')
textArr = text[0].split(' ')

price = int(re.sub('\D+', '', textArr[0]))
viewdata[0].update({'price': price})

address = ' '.join(textArr[1:]).strip()
viewdata[0].update({'address': address})

tel = text[1].strip()
viewdata[0].update({'tel': tel})

link = text[2].strip()
viewdata[0].update({'link': link})

viewdata[:1]

[{'title': 'Old Oak Tap BLT',
  'price': 10,
  'address': '2109 W. Chicago Ave.',
  'tel': '773-772-0406',
  'link': 'theoldoaktap.com'}]

In [122]:
listdata[:2]

[{'rank': 1,
  'menu': 'BLT',
  'cafe': 'Old Oak Tap',
  'title': 'Old Oak Tap BLT',
  'link': 'https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/'},
 {'rank': 2,
  'menu': 'Fried Bologna',
  'cafe': 'Au Cheval',
  'title': 'Au Cheval Fried Bologna',
  'link': 'https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Au-Cheval-Fried-Bologna/'}]

In [166]:
viewdata = []

for i, item in enumerate(listdata):

    viewdata.append({})

    # if i != 5:
    # if i > 9:
    #     continue

    web_url = item['link']

    print(i, web_url)

    with urllib.request.urlopen(web_url) as response:

        html = response.read()
        soup = BeautifulSoup(html, 'html.parser')

        # Title
        elTitle = soup.select_one('#page h1.headline')
        elTitleText = elTitle.get_text(strip=True)
        elTitleText = re.sub('^[0-9.]{1,}', '', elTitleText).strip()
        viewdata[i].update({'title': elTitleText})

        # Addy
        elAddy = soup.select_one('.addy')

        text = elAddy.em.get_text(strip=True).split(',')
        textArr = text[0].split(' ')
        textLen = len(text)

        price = int(re.sub('\D+', '', textArr[0]))
        viewdata[i].update({'price': price})

        address = ' '.join(textArr[1:]).strip()
        viewdata[i].update({'address': address})

        if textLen > 1:
            tel = text[1].strip()
            viewdata[i].update({'tel': tel})

        if textLen > 2:
            link = text[2].strip()
            viewdata[i].update({'link': link})

viewdata[:2]

0 https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/
1 https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Au-Cheval-Fried-Bologna/
2 https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Xoco-Woodland-Mushroom/
3 https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Als-Deli-Roast-Beef/
4 https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Publican-Quality-Meats-PB-L/
5 http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Hendrickx-Belgian-Bread-Crafter-Belgian-Chicken-Curry-Salad/
6 https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Acadia-Lobster-Roll/
7 https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Birchwood-Kitchen-Smoked-Salmon-Salad/
8 https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-San

[{'title': 'Old Oak Tap BLT',
  'price': 10,
  'address': '2109 W. Chicago Ave.',
  'tel': '773-772-0406',
  'link': 'theoldoaktap.com'},
 {'title': 'Au Cheval Fried Bologna',
  'price': 9,
  'address': '800 W. Randolph St.',
  'tel': '312-929-4580',
  'link': 'aucheval.tumblr.com'}]

In [94]:
listdata[:1]

[{'rank': 1,
  'menu': 'BLT',
  'cafe': 'Old Oak Tap',
  'title': 'Old Oak Tap BLT',
  'link': 'https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/'}]

In [174]:
df1 = pd.DataFrame(listdata)
df2 = pd.DataFrame(viewdata)
df = pd.merge(df1, df2, on='title', how='outer')

# Drop and Replace
# df.drop(['title_y', 'writer'], inplace=True, axis=1)
df.rename(columns={
    'link_x': 'link',
    'link_y': 'domain',
}, inplace=True)

# Reorder
df = df[['rank', 'menu', 'cafe', 'title', 'price', 'address', 'tel', 'domain', 'link']]

df.head()

,rank,menu,cafe,title,price,address,tel,domain,link
0,1.0,BLT,Old Oak Tap,Old Oak Tap BLT,10.0,2109 W. Chicago Ave.,773-772-0406,theoldoaktap.com,https://www.chicagomag.com/Chicago-Magazine/No...
1,2.0,Fried Bologna,Au Cheval,Au Cheval Fried Bologna,9.0,800 W. Randolph St.,312-929-4580,aucheval.tumblr.com,https://www.chicagomag.com/Chicago-Magazine/No...
2,3.0,Woodland Mushroom,Xoco,Xoco Woodland Mushroom,950.0,445 N. Clark St.,312-334-3688,rickbayless.com,https://www.chicagomag.com/Chicago-Magazine/No...
3,4.0,Roast Beef,Al’s Deli,Al’s Deli Roast Beef,940.0,914 Noyes St.,Evanston,847-475-9400,https://www.chicagomag.com/Chicago-Magazine/No...
4,5.0,PB&L,Publican Quality Meats,Publican Quality Meats PB&L,10.0,825 W. Fulton Mkt.,312-445-8977,publicanqualitymeats.com,https://www.chicagomag.com/Chicago-Magazine/No...


In [175]:
df.to_csv('04_chicago.csv', sep=',', encoding='utf8')

## 3-8 맛집 위치를 지도에 표시하기

```
conda install -c conda-forge folium
conda install -c conda-forge googlemaps
```

In [ ]:
import folium
import googlemaps

In [177]:
df = pd.read_csv('04_chicago.csv', index_col=0)
df.head()

,rank,menu,cafe,title,price,address,tel,domain,link
0,1.0,BLT,Old Oak Tap,Old Oak Tap BLT,10.0,2109 W. Chicago Ave.,773-772-0406,theoldoaktap.com,https://www.chicagomag.com/Chicago-Magazine/No...
1,2.0,Fried Bologna,Au Cheval,Au Cheval Fried Bologna,9.0,800 W. Randolph St.,312-929-4580,aucheval.tumblr.com,https://www.chicagomag.com/Chicago-Magazine/No...
2,3.0,Woodland Mushroom,Xoco,Xoco Woodland Mushroom,950.0,445 N. Clark St.,312-334-3688,rickbayless.com,https://www.chicagomag.com/Chicago-Magazine/No...
3,4.0,Roast Beef,Al’s Deli,Al’s Deli Roast Beef,940.0,914 Noyes St.,Evanston,847-475-9400,https://www.chicagomag.com/Chicago-Magazine/No...
4,5.0,PB&L,Publican Quality Meats,Publican Quality Meats PB&L,10.0,825 W. Fulton Mkt.,312-445-8977,publicanqualitymeats.com,https://www.chicagomag.com/Chicago-Magazine/No...


In [ ]:
key_fd = open('googlemaps_key.txt', mode='r')
gmaps_key = key_fd.read(100)
key_fd.close()

In [ ]:
gmaps_key = googlemaps
gmaps = googlemaps.Client(key=gmaps_key)